In [3]:
!pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
'''
'…', '...', '·' 제거 후 strip() 적용
'''
file_path = "../data/train_origin.csv"
df = pd.read_csv(file_path)

def clean_text(text):
    # Replace '… ', '...', and '·' with ' ' and then apply strip()
    cleaned_text = text.replace('…', ' ').replace('...', ' ').replace('·', ' ').strip()
    return cleaned_text

df['text'] = df['text'].apply(clean_text)

output_file_path = '../data/train.csv'
df.to_csv(output_file_path, index=False)